In [ ]:
import numpy as np
import cv2
from skimage.segmentation import slic, mark_boundaries
from skimage.future import graph
from scipy.ndimage import gaussian_filter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [ ]:
def create_superpixels(image, num_segments=100):
    # Apply SLIC algorithm
    segments = slic(image, n_segments=num_segments, compactness=10, start_label=1)
    return segments


In [ ]:
def calculate_similarity(superpixel1, superpixel2):
    # Calculate the mean color of each superpixel
    mean1 = np.mean(superpixel1, axis=(0, 1))
    mean2 = np.mean(superpixel2, axis=(0, 1))
    # Calculate the Euclidean distance between the mean colors
    similarity = np.linalg.norm(mean1 - mean2)
    return similarity

def merge_superpixels(image, segments, similarity_threshold=20):
    # Create a region adjacency graph (RAG)
    rag = graph.rag_mean_color(image, segments, mode='similarity')
    
    def merge_func(graph, src, dst):
        graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
        graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
        graph.nodes[dst]['labels'].update(graph.nodes[src]['labels'])
    
    def weight_func(graph, src, dst, n):
        diff = graph.nodes[dst]['total color'] / graph.nodes[dst]['pixel count'] - graph.nodes[src]['total color'] / graph.nodes[src]['pixel count']
        return {'weight': np.linalg.norm(diff)}
    
    labels = graph.merge_hierarchical(segments, rag, thresh=similarity_threshold, rag_copy=False,
                                      in_place_merge=True, merge_func=merge_func, weight_func=weight_func)
    return labels


In [ ]:
def assess_noise_level(superpixel):
    # Calculate the variance in color within the superpixel
    noise_level = np.var(superpixel)
    return noise_level

def split_noisy_superpixels(image, segments, noise_threshold=100):
    new_segments = np.copy(segments)
    for segment_label in np.unique(segments):
        superpixel = image[segments == segment_label]
        noise_level = assess_noise_level(superpixel)
        if noise_level > noise_threshold:
            mask = (segments == segment_label)
            blurred = gaussian_filter(image, sigma=2)
            edges = cv2.Canny((blurred * 255).astype(np.uint8), 100, 200)
            new_segments[mask] = segment_label + 1  # Increment segment label to split
    return new_segments


In [ ]:
def extract_features(superpixel):
    # Calculate the mean color of the superpixel
    mean_color = np.mean(superpixel, axis=(0, 1))
    return mean_color

def cluster_small_superpixels(image, segments, small_size_threshold=50):
    def extract_features(superpixel):
        return np.mean(superpixel, axis=(0, 1))
    
    new_segments = np.copy(segments)
    small_superpixels = [s for s in np.unique(segments) if np.sum(segments == s) < small_size_threshold]
    
    features = [extract_features(image[segments == s]) for s in small_superpixels]
    features = np.array(features)
    
    if len(features) > 1:
        kmeans = KMeans(n_clusters=min(2, len(features)), random_state=0).fit(features)
        for i, superpixel_label in enumerate(small_superpixels):
            new_segments[segments == superpixel_label] = kmeans.labels_[i] + 1
    return new_segments


v2


In [2]:
import numpy as np
import cv2
from skimage.segmentation import slic
from skimage.color import rgb2lab
from scipy.stats import multivariate_normal

def compute_superpixels(image, n_segments=100, compactness=10):
    lab_image = rgb2lab(image)
    segments = slic(lab_image, n_segments=n_segments, compactness=compactness, start_label=1)
    return segments

def compute_color_model(image, segments, n_segments):
    height, width, _ = image.shape
    lab_image = rgb2lab(image)

    superpixel_colors = []
    for segment_id in range(1, n_segments + 1):
        mask = (segments == segment_id)
        color_values = lab_image[mask]
        superpixel_colors.append(color_values)

    means = [np.mean(colors, axis=0) for colors in superpixel_colors]
    covariances = [np.cov(colors, rowvar=False) for colors in superpixel_colors]

    return means, covariances

def bayesian_segmentation(image, segments, means, covariances, n_segments):
    height, width, _ = image.shape
    lab_image = rgb2lab(image)
    bayesian_segments = np.zeros((height, width), dtype=np.int32)

    for y in range(height):
        for x in range(width):
            pixel = lab_image[y, x]
            max_prob = -np.inf
            best_segment = -1

            for segment_id in range(n_segments):
                mean = means[segment_id]
                cov = covariances[segment_id]
                prob = multivariate_normal.pdf(pixel, mean=mean, cov=cov)

                if prob > max_prob:
                    max_prob = prob
                    best_segment = segment_id + 1

            bayesian_segments[y, x] = best_segment

    return bayesian_segments

def main(image_path, n_segments=100, compactness=10):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    segments = compute_superpixels(image, n_segments, compactness)
    means, covariances = compute_color_model(image, segments, n_segments)
    bayesian_segments = bayesian_segmentation(image, segments, means, covariances, n_segments)

    return bayesian_segments

In [4]:
import os
import matplotlib.pyplot as plt
directory='train_images'
image_paths=[]
for img_dir in os.listdir(directory):
    image_paths.append( os.path.join(directory, img_dir))

print(image_paths)

for image_path in image_paths:
    segmented_image = main(image_path)

    plt.figure(10,10)
    plt.imshow(segmented_image)
    plt.axis(False)
    plt.title('Segmented Image')
    plt.show()

['train_images\\IMG-20240618-WA0003.jpg', 'train_images\\IMG-20240618-WA0004.jpg', 'train_images\\IMG-20240618-WA0005.jpg', 'train_images\\IMG-20240618-WA0006.jpg', 'train_images\\IMG-20240618-WA0007.jpg', 'train_images\\IMG-20240618-WA0008.jpg', 'train_images\\IMG-20240618-WA0009.jpg', 'train_images\\IMG-20240618-WA0010.jpg', 'train_images\\IMG-20240618-WA0013.jpg', 'train_images\\IMG-20240618-WA0014.jpg', 'train_images\\IMG-20240618-WA0015.jpg', 'train_images\\IMG-20240618-WA0016.jpg', 'train_images\\IMG-20240618-WA0017.jpg', 'train_images\\IMG-20240618-WA0018.jpg', 'train_images\\IMG-20240618-WA0019.jpg', 'train_images\\IMG-20240618-WA0020.jpg', 'train_images\\IMG-20240618-WA0021.jpg', 'train_images\\IMG-20240618-WA0022.jpg', 'train_images\\IMG-20240618-WA0023.jpg', 'train_images\\IMG-20240618-WA0024.jpg', 'train_images\\IMG-20240618-WA0025.jpg', 'train_images\\IMG-20240618-WA0026.jpg', 'train_images\\IMG-20240618-WA0027.jpg', 'train_images\\IMG-20240618-WA0028.jpg']


c:\waste-management\Version 3\environ\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\waste-management\Version 3\environ\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\waste-management\Version 3\environ\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\harsh\AppData\Local\Temp\ipykernel_18748\2673622138.py:23: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariances = [np.cov(colors, rowvar=False) for colors in superpixel_colors]
c:\waste-management\Version 3\environ\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\waste-management\Version 3\environ\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in

ValueError: array must not contain infs or NaNs